<a href="https://colab.research.google.com/github/zhibinz2/Neural-Networks-and-Machine-Learning/blob/master/ZHIBIN_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load files**

load from GitHub

(Looks like this work. But the data imported is not indexable?)


https://github.com/zhibinz2/Neural-Networks-and-Machine-Learning/raw/master/AwakeSamples.npy

AwakeSamples.npy contains 200 spectra samples of 51 by 91 matrices.

https://github.com/zhibinz2/Neural-Networks-and-Machine-Learning/raw/master/SedatedSamples.npy

SedatedSamples.npy contains 200 spectra samples of 51 by 91 matrices.

https://github.com/zhibinz2/Neural-Networks-and-Machine-Learning/raw/master/data.npy

data.npy contains 4000 sepctra samples. 

The first 200 samples are from AwakeSamples.npy

The second 200 samples are from SedatedSamples.npy



In [1]:
import numpy as np
!wget https://github.com/zhibinz2/Neural-Networks-and-Machine-Learning/raw/master/data.npy
data=np.load('data.npy')

--2020-12-06 05:33:04--  https://github.com/zhibinz2/Neural-Networks-and-Machine-Learning/raw/master/data.npy
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zhibinz2/Neural-Networks-and-Machine-Learning/master/data.npy [following]
--2020-12-06 05:33:04--  https://raw.githubusercontent.com/zhibinz2/Neural-Networks-and-Machine-Learning/master/data.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14851328 (14M) [application/octet-stream]
Saving to: ‘data.npy’

data.npy            100%[===================>]  14.16M  90.8MB/s    in 0.2s    

2020-12-06 05:33:04 (90.8 MB/s) - ‘data.npy’ saved [1485

In [2]:
import tqdm

import torch
from torch import optim
from torch import nn


import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torchvision import models, transforms
from torch.autograd import Variable

import pandas as pd
from PIL import Image
import requests
import io
import random, time, sys
from collections import namedtuple

import matplotlib.pyplot as plt

**Loading data**

In [5]:
data=np.expand_dims(data, axis=1)

train_data=data[0:300]
test_data=data[300:400]

import numpy.matlib
A=numpy.matlib.repmat(np.array([1, 0]),200,1)
B=numpy.matlib.repmat(np.array([0, 1]),200,1)
targets=np.concatenate((A,B),axis=0)

train_targets=targets[0:300]
test_targets=targets[300:400]

In [6]:
from torch.utils.data import Dataset
class MyTrainingDataset(Dataset):
    def __init__(self, root, transform=None, target_transform=None):
        self.root=root
        self.transform=transform
        self.target_transform=target_transform
        self.data=np.load(self.root[0])
        self.targets=np.load(self.root[1])
        
    def __len__(self):
        return len(self.data)
 
    def __getitem__(self, idx):
        sample=self.data[idx] 
        target=self.targets[idx]
        
        if self.transform:
            sample=self.transform(sample)
        if self.target_transform:
            target=self.target_transform(target)
            
        return sample, target

In [7]:
np.save('train_data.npy',train_data)
np.save('train_targets.npy',train_targets)
train_set=MyTrainingDataset(root=['train_data.npy', 'train_targets.npy'])

In [9]:
len(train_set[:][0])

300

In [11]:
np.save('test_data.npy',test_data)
np.save('test_targets.npy',test_targets)
test_set=MyTrainingDataset(root=['test_data.npy', 'test_targets.npy'])

In [12]:
test_set[0]

(array([[[4.19848455e-33, 2.31738664e-34, 2.56091811e-35, ...,
          2.78077053e-33, 1.74752529e-34, 4.13098596e-34],
         [3.51048667e-02, 4.09036823e-02, 3.37940482e-02, ...,
          4.09362925e-02, 3.23063199e-03, 2.88013818e-03],
         [1.00921740e-02, 1.53705540e-02, 2.73268636e-02, ...,
          3.83508883e-02, 2.83268583e-02, 2.81297980e-03],
         ...,
         [5.89317437e-06, 5.45644652e-05, 4.34597108e-05, ...,
          6.07942760e-06, 4.56696489e-05, 7.46810044e-07],
         [2.66073669e-04, 4.28187175e-05, 2.42064579e-05, ...,
          6.31677412e-05, 1.84963198e-05, 9.11167324e-06],
         [6.51550338e-06, 2.33473451e-06, 1.07256893e-06, ...,
          3.68026132e-05, 1.70397657e-06, 7.66378008e-06]]]), array([0, 1]))

In [13]:
len(test_set)

100

**Train_loader and test_loader**

In [14]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=100, shuffle=False)

In [15]:
len(train_loader)

3

**4_fcn_single_layer**

In [22]:
net = torch.nn.Sequential(torch.nn.Linear(4641, 100),torch.nn.Sigmoid(),torch.nn.Linear(100,2),
                                  torch.nn.Sigmoid())

In [23]:
mse_loss = torch.nn.MSELoss()
opt = torch.optim.Adam(net.parameters(), lr=1e-3)

In [24]:
def train_step(x, t, net, opt_fn, loss_fn):
    y = net(x)
    loss = loss_fn(y, t)
    loss.backward()
    opt_fn.step()
    opt_fn.zero_grad()
    return loss

In [26]:
x,t = next(iter(train_loader))

In [27]:
len(x)

100

In [28]:
train_step(x.float().view(-1,4641), t.float(), net, opt, mse_loss)

tensor(0.2610, grad_fn=<MseLossBackward>)

In [29]:
# Make prediction
for x,t in iter(train_loader):
  y = net(x.float().view(-1,4641))

In [30]:
len(y)

100

In [31]:
y.detach().numpy()

array([[0.5259743 , 0.54809207],
       [0.5313709 , 0.54179496],
       [0.5280322 , 0.5462043 ],
       [0.5321499 , 0.5400611 ],
       [0.5260596 , 0.5487482 ],
       [0.5275098 , 0.5435929 ],
       [0.5257776 , 0.5485522 ],
       [0.5266969 , 0.5476196 ],
       [0.5294443 , 0.5433193 ],
       [0.52686065, 0.54731697],
       [0.5244052 , 0.55155635],
       [0.52856576, 0.5450557 ],
       [0.5309974 , 0.5429959 ],
       [0.5236258 , 0.5527432 ],
       [0.5307731 , 0.5415578 ],
       [0.5244052 , 0.55155635],
       [0.5240127 , 0.55258864],
       [0.5346745 , 0.5368305 ],
       [0.5265103 , 0.5479601 ],
       [0.53248197, 0.54169565],
       [0.52422315, 0.5520909 ],
       [0.5289021 , 0.5419559 ],
       [0.53874654, 0.53404766],
       [0.5318042 , 0.53935355],
       [0.52777517, 0.54651374],
       [0.5282473 , 0.5461381 ],
       [0.52945894, 0.5435227 ],
       [0.52817875, 0.54454494],
       [0.5275167 , 0.546785  ],
       [0.5280601 , 0.5457197 ],
       [0.

In [ ]:
y.argmax(1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0])

In [ ]:
t.float()

tensor([1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
        1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0.,
        0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1.,
        1., 0., 0., 0., 1., 0., 1., 0., 1., 1.])

In [ ]:
torch.mean((t.float()==y.float()).float())

tensor(0.)

In [ ]:
# Calculate accuracy
torch.mean((t.argmax(1) == y.argmax(1)).float())

IndexError: ignored

In [ ]:
(t == y.float()).float()

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

**7_cnn_mnist.ipynb**

In [ ]:
class LeNet(torch.nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = torch.nn.Dropout2d(0.25)
        self.dropout2 = torch.nn.Dropout2d(0.5)
        self.pool2 = torch.nn.MaxPool2d((2,2),(2,2))
        self.fc1 = torch.nn.Linear(9216, 128)
        self.fc2 = torch.nn.Linear(128, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = torch.relu(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

net = LeNet().cuda()

In [ ]:
mse_loss = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(net.parameters(), lr=1e-3)

In [ ]:
def train_step(x, t, net, opt_fn, loss_fn):
    y = net(x.cuda())
    loss = loss_fn(y, t.cuda())
    loss.backward()
    opt_fn.step()
    opt_fn.zero_grad()
    return loss

In [ ]:
x,t = next(iter(train_loader))

In [ ]:
train_step(x.float(), t.float(), net.cuda, opt, mse_loss)

TypeError: ignored

**8_nin_nist**

In [ ]:
class NIN(torch.nn.Module):
    def __init__(self):
        super(NIN, self).__init__()

        self.nin1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, 5, padding=2),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 32, 1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 32, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(3, stride=2),
            torch.nn.Dropout(.5)          
        )
        self.nin2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, 1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, 1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(3, stride=2),
            torch.nn.Dropout(.5)          
        )
        self.nin3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, 3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(128, 128, 1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(128, 2, 1),
            torch.nn.ReLU(),
        )
        self.pool = torch.nn.AvgPool2d(6, stride=1)  


    def forward(self, x):
        x = self.nin1(x)
        x = self.nin2(x)
        x = self.nin3(x)
        x = self.pool(x)
        return x.view(-1,10)

net = NIN().cuda()

In [ ]:
data, targets = next(iter(train_loader))

In [ ]:
net(data.cuda())

RuntimeError: ignored